# Import

In [9]:
from sys import maxsize #para imprimir arrays completos
import numpy as np
import pandas as pd

from sklearn import preprocessing #para normalizar datos
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import metrics

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Metodos

In [2]:
#metodo para devolver un subset de datos, filtrando por state_name
def getSubDataFrameByStateName(sourceDataframe, state_name):
    df_result = sourceDataframe[sourceDataframe['state_name'] == state_name]
    
    #dropeamos la columna state_name, ya que en el df result todos pertenecen a la misma zona
    df_result = df_result.drop(labels='state_name', axis = 1)
    
    #mostramos informacion del df result
    print('Sub data frame Info: \n')
    print(df_result.info())
    
    print('\n\n')
    
    print('Sub data frame Head: \n')
    print(df_result.head(5))
    return df_result

In [3]:
def analyzeCategoricalData(dataFrame, categoricalVariablesArray):
    for category in categoricalVariablesArray:
        print(dataFrame[category].value_counts())
        plt.bar(dataFrame[category].value_counts().index, dataFrame[category].value_counts().values, color='b',\
                alpha=0.5)
        plt.show()

In [4]:
def createDummiesForVariables(dataFrame, categoricalVariablesArray):
    for category in categoricalVariablesArray:
        serie = dataFrame[category]
        dummies = pd.get_dummies(serie, drop_first= True, prefix=category)
        dataFrame = pd.concat([dataFrame, dummies], axis=1)
    
    return dataFrame

In [5]:
#genera un dataframe auxiliar para mantener la relacion entre los las variables categoricas originales y los respectivos
#dummies generados
def generateAuxiliarDataFrameForDummies(sourceDataFrame, dummiesColsPrefix):
    filter_col = [col for col in sourceDataFrame if col.startswith(dummiesColsPrefix)]
    dummies = sourceDataFrame[filter_col]
    dummies = dummies.loc[:,~dummies.columns.duplicated()]
    dummies = dummies.drop_duplicates()
    return dummies

In [6]:
#metodo para generar una regresion lineal simple
def generateSimpleLinearRegression(X_train, X_test, y_train, y_test, cv):
    regresionLineal = LinearRegression()
    
    #normalizamos los datos de training
    stdScaler = StandardScaler()
    X_train_norm = stdScaler.fit_transform(X=X_train)
    X_test_norm =  stdScaler.transform(X=X_test)
    
    regresionLineal_model = regresionLineal.fit(X_train_norm, y_train)
    
    #print('Score regresión lineal sin k-fold:', regresionLineal_model.score(X_test_norm, y_test))
    #scores_cv = cross_validate(estimator=regresionLineal_model, X=X_test_norm, y=y_test, cv=cv, return_train_score=True)
    
    scores_cv = cross_val_score(regresionLineal_model, X_train_norm, y_train, cv=cv, scoring="neg_mean_squared_error")
    
    print('Score regresión lineal sin k-fold:', scores_cv)
    
    #vemos los coeficientes y el interceptor obtenidos
    print ('\n\n')
    print ('Intercept: ')    
    print (regresionLineal_model.intercept_)
    print ('\n\n')
    print ('Coeficientes: ')    
    print (regresionLineal_model.coef_)
    print ('\n\n')
    
    #intentamos predecir
    regresionLineal_predicciones = regresionLineal_model.predict(X_test_norm)        
    
    #pasamos la Serie correspondiente a y_test a un array, para poder comparar luego los errores entre lo predicho 
    #contra los valores reales
    y_test_array = y_test.array
    
    print ('MAE:', metrics.mean_absolute_error(y_test_array, regresionLineal_predicciones))
    print ('MSE:', metrics.mean_squared_error(y_test, regresionLineal_predicciones))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, regresionLineal_predicciones)))
    print ('R2:', metrics.r2_score(y_test, regresionLineal_predicciones))
    
    #generamos un data frame auxiliar con los valores reales, los predichos y la diferencia entre ellos
    df_regresion_lineal_prediccion_vs_real = pd.DataFrame({'Real': y_test, \
                                                       'Predichos': regresionLineal_predicciones.flatten()})
    df_regresion_lineal_prediccion_vs_real['Diferencia'] = df_regresion_lineal_prediccion_vs_real['Real'] - \
                                                        df_regresion_lineal_prediccion_vs_real['Predichos']
    
    #retorna el modelo generado, las predicciones y el data frame que compara los valores reales con los predichos
    return regresionLineal_model, regresionLineal_predicciones, df_regresion_lineal_prediccion_vs_real

In [7]:
#metodo para generar un Ridge CV
def generateRidgeCV(X_train, X_test, y_train, y_test, alphasArray, cv):
    
    lm_ridge = RidgeCV(alphas=alphasArray, normalize=False, cv=cv) 
    
    #normalizamos los datos de training
    stdScaler = StandardScaler()
    X_train_norm = stdScaler.fit_transform(X=X_train)
    X_test_norm =  stdScaler.transform(X=X_test)
    
    model_ridge = lm_ridge.fit(X_train_norm, y_train)
     
    print('Score Ridge CV:', model_ridge.score(X_test_norm, y_test))
    
    print('Alpha Ridge CV:', model_ridge.alpha_)
        
    print ('Coeficientes: ', model_ridge.coef_)
    
    
    #intentamos predecir
    RidgeCV_predicciones = model_ridge.predict(X_test_norm)

    #pasamos la Serie correspondiente a y_test a un array, para poder comparar luego los errores entre lo predicho 
    #contra los valores reales
    y_test_array = y_test.array
    
    print ('MAE:', metrics.mean_absolute_error(y_true=y_test_array, y_pred=RidgeCV_predicciones))
    print ('MSE:', metrics.mean_squared_error(y_test, RidgeCV_predicciones))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, RidgeCV_predicciones)))
    print ('R2:', metrics.r2_score(y_test, RidgeCV_predicciones))
        
    #generamos un data frame auxiliar con los valores reales, los predichos y la diferencia entre ellos
    df_ridge_prediccion_vs_real = pd.DataFrame({'Real': y_test, \
                                           'Predichos': RidgeCV_predicciones.flatten()})
    df_ridge_prediccion_vs_real['Diferencia'] = df_ridge_prediccion_vs_real['Real'] - \
                                                            df_ridge_prediccion_vs_real['Predichos']
    
    #retorna el modelo generado, las predicciones y el data frame que compara los valores reales con los predichos
    return model_ridge, RidgeCV_predicciones,df_ridge_prediccion_vs_real

In [8]:
#metodo para generar un Ridge CV
def generateLassoCV(X_train, X_test, y_train, y_test, alphasArray, cv):
    
    #normalizamos los datos de training
    stdScaler = StandardScaler()
    X_train_norm = stdScaler.fit_transform(X=X_train)
    X_test_norm =  stdScaler.transform(X=X_test)
    
    #ejecutamos el LassoCV, usando el cross validation definido anteriormente con 5 iteraciones
    model_lasso = LassoCV().fit(X_train_norm, y_train)
    scores_lasso = cross_val_score(model_lasso, X_train_norm, y_train, cv=cv, scoring='r2')
    dict(alpha=model_lasso.alpha_, scores=scores_lasso, mean_score=scores_lasso.mean(), zero_coefs=(model_lasso.coef_ == 0).sum())
    
    print ('\n\n')
    print ('Intercept: ')
    print ('\n\n')
    print (model_lasso.intercept_)
    print ('\n\n')
    print ('Coeficientes: ')
    print ('\n\n')
    print (model_lasso.coef_)
    print ('\n\n')
    
    #intentamos predecir
    LassoCV_predicciones = model_lasso.predict(X_test_norm)
    
      #pasamos la Serie correspondiente a y_test a un array, para poder comparar luego los errores entre lo predicho 
    #contra los valores reales
    y_test_array = y_test.array
    
    print ('MAE:', metrics.mean_absolute_error(y_true=y_test_array, y_pred=LassoCV_predicciones))
    print ('MSE:', metrics.mean_squared_error(y_test, LassoCV_predicciones))
    print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, LassoCV_predicciones)))
    print ('R2:', metrics.r2_score(y_test, LassoCV_predicciones))
    
    #generamos un data frame auxiliar con los valores reales, los predichos y la diferencia entre ellos
    df_lasso_prediccion_vs_real = pd.DataFrame({'Real': y_test, \
                                           'Predichos': LassoCV_predicciones.flatten()})
    df_lasso_prediccion_vs_real['Diferencia'] = df_lasso_prediccion_vs_real['Real'] - \
                                                            df_lasso_prediccion_vs_real['Predichos']
    df_lasso_prediccion_vs_real

        
    #retorna el modelo generado, las predicciones y el data frame que compara los valores reales con los predichos
    return model_lasso,LassoCV_predicciones, df_lasso_prediccion_vs_real